In [1]:
import requests
from pprint import pprint
import xarray as xr
import numpy
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
from harmony import BBox, Client, Collection, Request, LinkType
from harmony.config import Environment
import datetime as dt

#to make big figures
plt.rcParams['figure.figsize'] = 12, 6

In [2]:
#select L2 data 
dataset='podaac-ops-cumulus-protected/JASON_CS_S6A_L2_ALT_LR_RED_OST_NRT_F'
shortname='JASON_CS_S6A_L2_ALT_LR_RED_OST_NRT_F'

#timeframe of interest:
start_date = "2015-08-01"
end_date   = "2015-10-31"

#region of interest:
lonmin = -170.0
latmin = 65.0
lonmax = -120.0
latmax = 80.0

In [3]:
CMR_OPS = 'https://cmr.earthdata.nasa.gov/search'
collection_url = f'{CMR_OPS}/{"collections"}'

In [4]:
provider = 'POCLOUD'
response = requests.get(collection_url,
                        params={
                            'short_name': shortname,
                            'cloud_hosted': 'True',
                            'has_granules': 'True',
                            'provider': provider,
                            'page_size': 251,
                        },
                        headers={
                            'Accept': 'application/json'
                        }
                       )
collections = response.json()['feed']['entry']
for collection in collections:
    print(f'{collection["archive_center"]} {collection["dataset_id"]} {collection["id"]}')

NASA/JPL/PODAAC Sentinel-6A MF Jason-CS L2 P4 Altimeter Low Resolution (LR) NRT Reduced Ocean Surface Topography C1968980576-POCLOUD


In [5]:
concept_id='C1968980576-POCLOUD'
response = requests.get(collection_url, 
                        params={
                            'concept_id': concept_id,
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
response = response.json()

In [6]:
services = response['feed']['entry'][0]['associations']['services']
print(services)
service_url = "https://cmr.earthdata.nasa.gov/search/services"

['S2004184019-POCLOUD', 'S1962070864-POCLOUD']


In [7]:
service_response = requests.get(service_url, 
                        params={
                            'concept_id': services[0],
                            },
                        headers={
                            'Accept': 'application/vnd.nasa.cmr.umm_results+json'
                            }
                       )
service_response = service_response.json()

In [8]:
variables = response['feed']['entry'][0]['associations']['variables']
var_url = "https://cmr.earthdata.nasa.gov/search/variables"
var_list = []
for i in range(len(variables)):
    var_response = requests.get(var_url, 
                            params={
                                'concept_id': variables[i],
                                },
                            headers={
                                'Accept': 'application/vnd.nasa.cmr.umm_results+json'
                                }
                           )
    var_response = var_response.json()
    var_list.append(var_response['items'][0]['umm']['Name'])
pprint(var_list)

['/data_01/mean_dynamic_topography_acc',
 '/data_01/ku/swh_ocean_rms',
 '/data_01/rad_tb_238',
 '/data_01/rad_tmb_340',
 '/data_01/off_nadir_roll_angle_pf',
 '/data_01/c/iono_cor_gim',
 '/data_01/ku/rad_atm_cor_sig0_qual',
 '/data_01/ku/range_ocean_mle3_numval',
 '/data_01/ku/sig0_ocean_mle3_rms',
 '/data_01/c/range_ocean',
 '/data_01/c/index_first_20hz_measurement',
 '/data_01/rad_rain_flag',
 '/data_01/wind_speed_mod_u',
 '/data_01/ku/net_instr_cor_sig0_ocean',
 '/data_01/ku/index_first_20hz_measurement',
 '/data_01/off_nadir_pitch_angle_pf',
 '/data_01/ku/net_instr_cor_range_ocean_mle3',
 '/data_01/c/swh_ocean_numval',
 '/data_01/ku/range_ocean_numval',
 '/data_01/rad_land_frac_187',
 '/data_01/geoid',
 '/data_01/internal_tide',
 '/data_01/c/sig0_ocean_qual',
 '/data_01/c/model_instr_cor_range_ocean',
 '/data_01/ku/sig0_ocean_qual',
 '/data_01/mean_sea_surface_sol1_qual',
 '/data_01/c/net_instr_cor_range_ocean',
 '/data_01/ku/net_instr_cor_range_ocean',
 '/data_01/iono_cor_alt_mle3'

In [9]:
harmony_client = Client()

In [10]:
request = Request(
    collection=Collection(id=shortname),
    spatial=BBox(lonmin,latmin,lonmax,latmax),
    temporal={
        'start': dt.datetime(2021, 8, 1),
        'stop': dt.datetime(2021, 10, 31),
    },
)
request.is_valid()

True

In [11]:
job_id = harmony_client.submit(request)
job_id
harmony_client.status(job_id)

{'status': 'running',
 'message': 'CMR query identified 276 granules, but the request has been limited to process only the first 200 granules because of system constraints.',
 'progress': 0,
 'created_at': datetime.datetime(2021, 11, 19, 18, 20, 35, 951000, tzinfo=tzlocal()),
 'updated_at': datetime.datetime(2021, 11, 19, 18, 20, 35, 951000, tzinfo=tzlocal()),
 'request': 'https://harmony.earthdata.nasa.gov/JASON_CS_S6A_L2_ALT_LR_RED_OST_NRT_F/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?forceAsync=true&subset=lat(65.0%3A80.0)&subset=lon(-170.0%3A-120.0)&subset=time(%222021-08-01T00%3A00%3A00%22%3A%222021-10-31T00%3A00%3A00%22)',
 'num_input_granules': 200}

In [12]:
harmony_client.wait_for_processing(job_id, show_progress=True)

 [ Processing:   0% ] |                                                   | [/]


ProcessingFailedException: Failed

In [ ]:
data = harmony_client.result_json(job_id)
pprint(data)

In [ ]:
results = harmony_client.result_urls(job_id, link_type=LinkType.s3)
urls = list(results)
pprint(urls)